In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from sklearn import preprocessing
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
!wget https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
!tar -xvzf flower_photos.tgz

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]

# Transforms to be applied to Train-Test-Validation
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(means, stds)])

### Dataloader

In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, img_path):
        self.img_path = img_path

        self.data = []
        self.label = []
        for i in os.listdir(self.img_path):
            if os.path.isdir(self.img_path + "/" + i):
                for j in os.listdir(self.img_path + "/" + i):
                    self.data.append(self.img_path + "/" + i + "/" + j)
                    self.label.append(str(i))
        label_encoder = preprocessing.LabelEncoder()
        self.label = label_encoder.fit_transform(self.label)
        mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
        print(mapping)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = Image.open(self.data[idx])
        img = train_transforms(img)
        label = self.label[idx]
        label_tensor = torch.as_tensor(label, dtype=torch.long)
        return {'im': img, 'labels': label_tensor}

In [6]:
dataset = CustomImageDataset("./flower_photos")
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


### Model

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [8]:
model = Net().to(device)

### Optimizer and loss

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

### Train

In [10]:
for epoch in range(50):
    print("EPOCH:",epoch,end=" ")
    running_loss=0
    running_acc=0
   
    for i, data in enumerate(dataloader):
        inputs, labels = data['im'].to(device), data['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        result = torch.argmax(outputs,dim=1)
        running_loss+=loss.item()
        running_acc+=torch.mean((result==labels).type(torch.float))
        
        loss.backward()
        optimizer.step()
        
    else:
        train_loss=running_loss/len(dataloader)
        train_acc=running_acc/len(dataloader)
        
        print("Training Loss: {:.3f}".format(train_loss),end=" ") 
       
        print("Train Accuracy: {:.2f}%".format(train_acc.item()*100))


EPOCH: 0 Training Loss: 1.881 Train Accuracy: 25.57%
EPOCH: 1 Training Loss: 1.535 Train Accuracy: 30.12%
EPOCH: 2 Training Loss: 1.484 Train Accuracy: 33.63%
EPOCH: 3 Training Loss: 1.391 Train Accuracy: 40.03%
EPOCH: 4 Training Loss: 1.297 Train Accuracy: 44.73%
EPOCH: 5 Training Loss: 1.280 Train Accuracy: 45.51%
EPOCH: 6 Training Loss: 1.258 Train Accuracy: 46.52%
EPOCH: 7 Training Loss: 1.215 Train Accuracy: 49.45%
EPOCH: 8 Training Loss: 1.200 Train Accuracy: 50.14%
EPOCH: 9 Training Loss: 1.182 Train Accuracy: 52.15%
EPOCH: 10 Training Loss: 1.153 Train Accuracy: 52.39%
EPOCH: 11 Training Loss: 1.149 Train Accuracy: 53.89%
EPOCH: 12 Training Loss: 1.133 Train Accuracy: 54.64%
EPOCH: 13 Training Loss: 1.111 Train Accuracy: 55.69%
EPOCH: 14 Training Loss: 1.087 Train Accuracy: 56.22%
EPOCH: 15 Training Loss: 1.079 Train Accuracy: 57.53%
EPOCH: 16 Training Loss: 1.081 Train Accuracy: 56.26%
EPOCH: 17 Training Loss: 1.071 Train Accuracy: 57.99%
EPOCH: 18 Training Loss: 1.049 Train A